In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
from train import SimCLR_Train, SimCLR_new
from models import SimCLR_Model

In [ ]:
# credit: https://github.com/kuangliu/pytorch-cifar/issues/19
# for CIFAR-10 normalization values
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

batch_size_list = [256, 512, 1024, 2048, 4096]

# the maximum batchsize is 1024
batch_size = batch_size_list[2]

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [ ]:
# manually augment before training to save time
from augmentations import augment
aug1_list = []
aug2_list = []
for batch in train_loader:
    image, _ = batch
    aug1, aug2 = augment(image, augment_twice=True)
    aug1_list.append(aug1)
    aug2_list.append(aug2)

In [ ]:
# use adam/sgd since we don't have large batches
# possible config for hyperparams:
# temp = [0.1, 0.5, 1.0]
# lr = 1e-4
# epoch = [100, 200, 300]
# could add a cosine scheduler
# check here for more config: https://github.com/p3i0t/SimCLR-CIFAR10
resnet20 = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=False)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = SimCLR_Model(base_encoder=resnet20)
net = net.to(device)

In [ ]:
train_loss_list = SimCLR_new(net, device, batch_size, train_loader, num_epoch=100, augment1=aug1_list, augment2=aug2_list, temp=0.5, lr=1e-4, optim='adam', lr_scheduler='None')

In [ ]:
# linear evaluation code--rerun this block for linear eval

from train import Linear_Eval_Train
from models import Linear_Eval
resnet20 = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=False)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = SimCLR_Model(base_encoder=resnet20)
net = net.to(device)
net.load_state_dict(torch.load('SimCLR_new_batchsize=1024_lr=0.0001_optim=adam_temp=0.5_epoch=100.pt'))

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

batch_size = 512

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

base = net.base_encoder
linear_eval = Linear_Eval(base)
linear_eval = linear_eval.to(device)
loss_list, acc_list = Linear_Eval_Train(linear_eval, device, train_loader, num_epoch=100, lr=0.1, optim='sgd', lr_scheduler='cosine')

In [3]:
# linear evaluation code--rerun this block for linear eval

from train import Linear_Eval_Train
from models import Linear_Eval
resnet20 = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=False)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = SimCLR_Model(base_encoder=resnet20)
net = net.to(device)
net.load_state_dict(torch.load('SimCLR_new_batchsize=1024_lr=0.0001_optim=adam_temp=0.5_epoch=100.pt'))

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

batch_size = 512

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

base = net.base_encoder
linear_eval = Linear_Eval(base)
linear_eval = linear_eval.to(device)
loss_list, acc_list = Linear_Eval_Train(linear_eval, device, train_loader, num_epoch=100, lr=0.1, optim='sgd', lr_scheduler='cosine')

Using cache found in C:\Users\Koir/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master


Files already downloaded and verified
Files already downloaded and verified
Epoch  1


100%|██████████████████████████████████████████████████████████████████████████████████| 98/98 [00:06<00:00, 15.36it/s]


Training Loss: 2.153761148452759, Training Acc: 0.2139725387096405
Best Train Acc:  0.2139725387096405
Epoch  2


100%|██████████████████████████████████████████████████████████████████████████████████| 98/98 [00:04<00:00, 23.32it/s]


...

Training Loss: 1.7377357482910156, Training Acc: 0.37130630016326904
Best Train Acc:  0.37130630016326904
Epoch  99


100%|██████████████████████████████████████████████████████████████████████████████████| 98/98 [00:04<00:00, 23.59it/s]


Training Loss: 1.7370651960372925, Training Acc: 0.3701295256614685
Best Train Acc:  0.37130630016326904
Epoch  100


100%|██████████████████████████████████████████████████████████████████████████████████| 98/98 [00:04<00:00, 23.33it/s]

Training Loss: 1.7377065420150757, Training Acc: 0.36929625272750854
Best Train Acc:  0.37130630016326904


In [4]:
# test linear eval
linear_eval.eval()
test_acc = 0
for batch in test_loader:
    image, label = batch
    image, label = image.to(device), label.to(device)
    loss_func = nn.CrossEntropyLoss()
    actual_batch_size = image.shape[0]
    pred = linear_eval(image)
    loss = loss_func(pred, label)
    test_acc += torch.sum(torch.argmax(pred, dim=-1) == label) / actual_batch_size
test_acc /= len(test_loader)
print('Test Acc: ', test_acc.item())

Test Acc:  0.36190831661224365
